In [1]:
import requests
import json
from bs4 import BeautifulSoup as bs
import time
import random
import re
import pandas as pd
import os
import numpy as np

In [2]:
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords

import numpy as np
from tqdm import tqdm

%%time
from gensim.models.wrappers import FastText
model_fr = FastText.load_fasttext_format('/Users/nigula/input/fr/fr.bin')

%%time
from gensim.models.keyedvectors import FastTextKeyedVectors
model_ru= FastTextKeyedVectors.load('/Users/nigula/input/araneum_none_fasttextcbow_300_5_2018/araneum_none_fasttextcbow_300_5_2018.model')

In [3]:
def send_request_func(from_lang,to_lang, word, header_main, login, en_from_lang, en_to_lang):
    start = time.time()
    url = "https://context.reverso.net/перевод/" + from_lang + "-" + to_lang + "/" + word 
    print("going to find word in", url)
    response = requests.get(url, headers=header_main, data = login)
    response.encoding = 'utf-8' 
    #
    save_dir = os.path.join("/Users/nigula/LL/adjust_unigr_dict/lookup/reverso" + "_" + en_from_lang + "_" + en_to_lang,word + ".xls")
    file = open(save_dir, "wb")
    file.write(response.content)
    file.close()
    print("saved to directory", save_dir)
    soup = bs(response.text, 'html.parser') 
    time.sleep(0.01)
    print("request_time",time.time() - start) 
    return soup

In [6]:
def get_definitions_reverso(word, from_lang, to_lang, print_output = False):
    start = time.time()
    login = {'inUserName': 'n.babakov@lingualeo.com', 'inUserPass': '33vec33'}
    header_main = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    senseword_list = []
    land_dict = {"русский":"ru","французский":"fr", "английский":"en"}
    try:
        word_dir = os.path.join("/Users/nigula/LL/adjust_unigr_dict/lookup/reverso" + "_" + land_dict[from_lang] + "_" + land_dict[to_lang],word + ".xls")
        infile = open(word_dir,"r")
        print("found_word_in", word_dir)
        contents = infile.read()
        soup = bs(contents,'html.parser')
    except:
        soup = send_request_func(from_lang,to_lang, word, header_main, login, land_dict[from_lang], land_dict[to_lang])
    #print(soup.prettify())
    first_string_passed = False
    pos = ''
    for link in soup.find_all("a", attrs={"class" : "translation"}):
        #print(link.prettify())
        try:
            #print("||",link['data-pos'])
            pos = link['data-pos'][1:-1]
        except:
            pass
        if first_string_passed == True:
            sence_word = link.text.strip()
            if len(sence_word) >0:
                senseword_list.append({"sence_word":sence_word.lower(), "pos":pos})
            #print(sence_word)
        first_string_passed = True
        
    for link in soup.find_all("div", attrs={"class" : "translation"}):
        #print(link.prettify())
        try:
            #print("||",link['data-pos'])
            pos = link['data-pos'][1:-1]
        except:
            pass
        if first_string_passed == True:
            sence_word = link.text.strip()
            if len(sence_word) >0:
                senseword_list.append({"sence_word":sence_word.lower(), "pos":pos})
            #print(sence_word)
        first_string_passed = True
    if len (senseword_list) == 0:
        print("turn to alternative marks")
        for link in soup.find_all("div", attrs={"lang" : "fr"}):
            try:
                #print("||",link['data-pos'])
                pos = link['data-pos'][1:-1]
            except:
                pass
            sence_word = link.text.strip()
            senseword_list.append({"sence_word":sence_word.lower(), "pos":pos})
            #print(link.prettify())
    print("definiotns_time", time.time() - start)
    return senseword_list


get_definitions_reverso("Extra", "французский", "русский", print_output = False)



found_word_in /Users/nigula/LL/adjust_unigr_dict/lookup/reverso_fr_ru/Extra.xls
definiotns_time 0.11478376388549805


[{'sence_word': 'высшего сорта', 'pos': ''},
 {'sence_word': 'экстра', 'pos': 'adv'},
 {'sence_word': 'сверху', 'pos': 'adv'},
 {'sence_word': 'бонус', 'pos': 'n'},
 {'sence_word': 'крутой', 'pos': 'adj'},
 {'sence_word': 'пространства', 'pos': 'adj'},
 {'sence_word': 'первого сортов', 'pos': 'adj'},
 {'sence_word': 'классно', 'pos': 'adj'},
 {'sence_word': 'дополнительно', 'pos': 'adj'},
 {'sence_word': 'замечательная', 'pos': 'adj'},
 {'sence_word': 'дополнительный', 'pos': 'adj'},
 {'sence_word': 'отличный парень', 'pos': 'adj'}]

# LEMMATIZERS

In [7]:
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
lemmatizer = FrenchLefffLemmatizer()

def fr_lemmatize(word):
    lemma = lemmatizer.lemmatize(word)
    return lemma
def preprocess_words(words_list, lemmatizer_func):
    start = time.time()
    preprocessed_words = []
    for word in words_list:
        lemma = lemmatizer_func(word['sence_word'].lower())
        word['sence_word'] = lemma
    print("lemmat_time", time.time() - start)
        #print("lemma",lemma)
        #preprocessed_words.append(lemma)
    #return preprocessed_words
        

In [8]:

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
    
def get_nonstopwords_vectors_sum(lemm_list):
    orig_vector = np.zeros(shape = 100)
    for lemma in lemm_list:
        if lemma not in stopWords:
            #print (lemma)
            try:
                orig_vector += model[lemma]
            except:
                pass
    return orig_vector


def en_lemmatize(line):
    pos_tagged_ngramm = pos_tag(line.split())
    lemmatized_line_list = []
    for word_el in pos_tagged_ngramm:
        pos = get_wordnet_pos(word_el[1])
        if pos:
            lemma = lemmatizer.lemmatize(word_el[0], pos =pos)
        else:
            lemma = word_el[0]
        lemmatized_line_list.append(lemma)
    return ' '.join(lemmatized_line_list)

In [9]:
from pymystem3 import Mystem
text = "Красивая мама красиво мыла раму"
mystem = Mystem()
lemmas = mystem.lemmatize(text)
print(''.join(lemmas))

красивый мама красиво мыть рама



In [10]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
def ru_lemmatize(line):
    line_lemm = ''
    for word in line.split():
        lemma = morph.parse(word)[0]
        lemma = lemma.normal_form
        line_lemm += lemma + ' '
    line_lemm = line_lemm.strip()
    return line_lemm

ru_lemmatize("Красивая мама красиво мыла раму")

'красивый мама красиво мыло рам'

In [11]:
l1 = "maman"
l2 = 'maman famille'

def get_vectors_sum(lemm_line, lang):
    if lang == "французский": model = model_fr
    elif lang == "русский": model = MOD
    else:
        print("NO_MODEL_FOUND")
        return np.zeros(shape = 300)
    orig_vector = np.zeros(shape = 300)
    for lemma in lemm_line.split():
        #print(lemma)
        try:
            orig_vector += model[lemma]
        except:
            pass
    return orig_vector

def find_lines_similarity(line_1, line_2, lang):
    l1_vect = get_vectors_sum(line_1, lang)
    l2_vect = get_vectors_sum(line_2, lang)
    cos_sim = cosine_similarity(l1_vect.reshape(1, -1), l2_vect.reshape(1, -1))[0][0]
    #print(cos_sim)
    return cos_sim

find_lines_similarity(l1, l2, "французский")
#find_lines_similarity(l1, l2, "русский")



NameError: name 'model_fr' is not defined

In [12]:
def compare_pos(pos1, pos2):
    if pos1 == pos2:
        #print("abs eq", pos1, pos2)
        return True
    elif pos1 in pos2 or pos2 in pos1:
        #print("inclusive eq", pos1, pos2)
        return True
    #print("no eq", pos1, pos2)
    return False
def equate_empty_pos(pos1, pos2):
    if pos1 == '': 
        pos1 = pos2
        #print("pos1 eqauted")
    elif pos2 == '': 
        pos2 = pos1
        #print("pos2 eqauted")
    return pos1, pos2

In [13]:
def find_lines_inclusive_similarity(line_1,line_2):
    if line_1 in line_2: return True
    elif line_2 in line_1: return True
    else: return False
    
    
find_lines_inclusive_similarity("chauffer","réchauffer")

True

In [ ]:
#handle indirect similarity
handled_words = []
for word_1 in list_1:
    for word_2 in list_2:
        word_1['pos'], word_2['pos'] = equate_empty_pos(word_1['pos'], word_2['pos'])
        pos_eq = compare_pos(word_1['pos'],word_2['pos'])
        if pos_eq == True and word_1['sence_word'] != word_2['sence_word'] and word_1['sence_word'] not in handled_words and word_2['sence_word'] not in handled_words:
            sim = find_lines_similarity(word_1['sence_word'], word_2['sence_word'], lists_lang)
            if sim > 0.9:
                print('COS_SIM_WORKED', word_1['sence_word'], word_2['sence_word'])
                if word_1['sence_word'] in cos_sim_dict:
                    handled_words.append(word_2['sence_word'])
                    cos_sim_dict[word_1['sence_word']].append(word_2['sence_word'])
                   # overall_intersection[word_1_index] += '| ' + word_2

                elif word_2['sence_word'] in cos_sim_dict:
                     handled_words.append(word_1['sence_word'])
                     cos_sim_dict[word_2['sence_word']].append(word_1['sence_word'])
                else:
                    cos_sim_dict[word_1['sence_word']] = [word_2['sence_word']]

In [14]:
def get_two_target_lang_lists_intersection(list_1, list_2, lists_lang):
    start = time.time()
    overall_intersection = []
    cos_sim_dict ={}
    #overall_intersection
    for word_1 in list_1:
        for word_2 in list_2:
            #print(word_1, word_2)
            word_1['pos'], word_2['pos'] = equate_empty_pos(word_1['pos'], word_2['pos'])
            pos_eq = compare_pos(word_1['pos'],word_2['pos'])  
            if pos_eq == True and word_1['sence_word'] == word_2['sence_word']:
                #overall_intersection.append(word_1)
                cos_sim_dict[word_1['sence_word']] = []
                    
    #GETET INTERSECTION LIST
    for first_word in cos_sim_dict.keys():
        intersect_element = first_word
        for similar_word in cos_sim_dict[first_word]:
            intersect_element += " | " + similar_word
        overall_intersection.append(intersect_element)
    print(time.time() - start, "passed get_two_target_lang_lists_intersection")
    return overall_intersection

fr_proc_1 = [{'sence_word': 'réchauffer', 'pos': 'v'}, {'sence_word': 'chaud', 'pos': 'v'}]

fr_proc_2 = [{'sence_word': 'chaud', 'pos': 'adj'},
 {'sence_word': 'chaleureux', 'pos': 'adj'},
 {'sence_word': 'tiède', 'pos': 'adj'},
 {'sence_word': 'doux', 'pos': 'adj'},
 {'sence_word': 'bon', 'pos': 'adj/nm'},
 {'sence_word': 'cordial', 'pos': 'adj'},
 {'sence_word': 'réchauffer', 'pos': 'v'},
 {'sence_word': 'chauffer', 'pos': 'v'},
 {'sence_word': 'échauffer', 'pos': 'v'},
 {'sence_word': 'se réchauffer', 'pos': 'v'},
 {'sence_word': 'tiédir', 'pos': 'v'},
 {'sence_word': 'chaleur', 'pos': 'nf'},
 {'sence_word': 'accueil', 'pos': 'nm'},
 {'sence_word': 'chaleureusement', 'pos': 'adv'},
 {'sence_word': 'warm', 'pos': 'adv'},
 {'sence_word': 'accueillante', 'pos': 'adv'},
 {'sence_word': 'conviviale', 'pos': 'adv'}]
get_two_target_lang_lists_intersection(fr_proc_1, fr_proc_2, "fr")      
            

2.9087066650390625e-05 passed get_two_target_lang_lists_intersection


['réchauffer']

In [15]:
def get_cross_translate_reverso(word_lang_1, lang_1, word_lang_2, lang_2, target_lang, lemmatizer_func):
    lang_1_to_target_words_dict = get_definitions_reverso(word_lang_1, lang_1, target_lang, print_output=True)
    preprocess_words(lang_1_to_target_words_dict, lemmatizer_func)
    
    lang_2_to_target_words_dict = get_definitions_reverso(word_lang_2, lang_2, target_lang, print_output=True)
    preprocess_words(lang_2_to_target_words_dict, lemmatizer_func)
    
    print("lang_1_to_target_words_dict", lang_1_to_target_words_dict,"\n")
    print("lang_2_to_target_words_dict", lang_2_to_target_words_dict)
    
    intersected_elements = get_two_target_lang_lists_intersection(lang_1_to_target_words_dict, lang_2_to_target_words_dict,
                                                                 target_lang)
    
    if len(intersected_elements) == 0:intersected_elements.append("no_equality")
    return intersected_elements

get_cross_translate_reverso("warm", "английский", "согревать","русский", "французский", fr_lemmatize)


found_word_in /Users/nigula/LL/adjust_unigr_dict/lookup/reverso_en_fr/warm.xls
definiotns_time 0.11221694946289062
lemmat_time 0.0005366802215576172
found_word_in /Users/nigula/LL/adjust_unigr_dict/lookup/reverso_ru_fr/согревать.xls
definiotns_time 0.2977890968322754
lemmat_time 6.198883056640625e-05
lang_1_to_target_words_dict [{'sence_word': 'chaud', 'pos': 'adj'}, {'sence_word': 'chaleureux', 'pos': 'adj'}, {'sence_word': 'tiède', 'pos': 'adj'}, {'sence_word': 'doux', 'pos': 'adj'}, {'sence_word': 'bon', 'pos': 'adj/nm'}, {'sence_word': 'cordial', 'pos': 'adj'}, {'sence_word': 'réchauffer', 'pos': 'v'}, {'sence_word': 'chauffer', 'pos': 'v'}, {'sence_word': 'échauffer', 'pos': 'v'}, {'sence_word': 'se réchauffer', 'pos': 'v'}, {'sence_word': 'tiédir', 'pos': 'v'}, {'sence_word': 'chaleur', 'pos': 'nf'}, {'sence_word': 'accueil', 'pos': 'nm'}, {'sence_word': 'chaleureusement', 'pos': 'adv'}, {'sence_word': 'warm', 'pos': 'adv'}, {'sence_word': 'accueillante', 'pos': 'adv'}, {'sence_w

['réchauffer']

In [16]:
def get_multilang_from_df(df, lookup_from, lookup_to, target_lang, target_lang_lemmatizer_func):
    land_dict = {"русский":"ru","французский":"fr", "английский":"en"}
    eng_word_list = []
    lang_1_word_list = []
    lang_2_word_list = []
    eng_word = 'first_word'
    save_triple_path = land_dict[lookup_from] + '_' + land_dict[lookup_to] + '_' + land_dict[target_lang]
    
    start = False
    
    #for df_ind in tqdm(range(len(df))):
    count = 0 
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        if eng_word != row["word"]: print("\n************************************\n")
        eng_word = row["word"]
        #if eng_word == "trajet": start = True
        #if start == True:
        loc_1_word = row["local_word"]
        print(eng_word, loc_1_word)
        sense_intersect = get_cross_translate_reverso(eng_word, lookup_from, loc_1_word, lookup_to, target_lang, target_lang_lemmatizer_func)
        #sense_intersect = get_cross_translate_yand(eng_word, "en", loc_1_word,"ru", "fr", fr_lemmatize)
        print("\n SENSE_INTERSECTION ", sense_intersect)
        for sense_intersect_element in sense_intersect:
            eng_word_list.append(eng_word)
            lang_1_word_list.append(loc_1_word)
            lang_2_word_list.append(sense_intersect_element)


        if count % 10 == 0 and count != 0:

            save_path = os.path.join("/Users/nigula/LL/adjust_unigr_dict/reverse_connected_dicts/cross_barbos",
                         save_triple_path, str(count) + ".csv")
            print(save_path)
            data = pd.DataFrame(list(zip(eng_word_list, lang_1_word_list,lang_2_word_list)),
                        columns =[land_dict[lookup_from], land_dict[lookup_to], land_dict[target_lang]])
            data.to_csv(save_path)
        count += 1
            
    data = pd.DataFrame(list(zip(eng_word_list, lang_1_word_list,lang_2_word_list)),
                        columns =[land_dict[lookup_from], land_dict[lookup_to], land_dict[target_lang]])
    
    return data

#df_din = get_multilang_from_df(df_en_ru_yandex, "английский", "русский", "французский", fr_lemmatize)
#df_din.head()

In [ ]:
en-ru >> fr ready_res
ru-en >> fr

en-fr >>ru ready_res
fr-en >>ru ready_res

ru-fr>>en ready_res
fr-ru>>en ready_res